# * VINSIGHT : Data Issue

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Issue History

### Prepaid Active Caller 30D Rolling --> Monitoring...

In [39]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY --, CENTER, PRODUCT_GRP, METRIC_GRP
        , METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME, ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH --, TARGET_SNAP
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    --WHERE METRIC_CD = 'TB1S000600' --Prepaid Active Caller : TMH
    WHERE METRIC_CD IN (
         'B1S000600' --Prepaid Active Caller
         , 'TB1S000600' --Prepaid Active Caller : TMH
         , 'DB1S000600' --Prepaid Active Caller : DTAC
         )
    AND AREA_TYPE IN ('P', 'G')
    --AND AREA_CD = 'P'
    AND TM_KEY_DAY >= {v_start_date} 
    --ORDER BY TM_KEY_DAY, CENTER, PRODUCT_GRP, METRIC_GRP, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    # txn_df = chk_src_df.copy()
    # # txn_df = txn_df.fillna(0, inplace=True)
    # # txn_df = txn_df.replace(np.nan, None)
    # mod_col_list = txn_df.iloc[:, 6:8].columns.tolist()
    # for col in mod_col_list:
    #     txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    # print(f'\n{txn_df.to_string(max_cols=10)}') #max_rows=1000
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-23, 16:26:17
   -> v_start_date: 20240101


In [40]:
''' Automate Month Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

In [41]:
''' B1S000600 : Prepaid Active Caller 30D Rolling '''

v_metric_cd = 'B1S000600'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B1S000600_day_df = chk_src_df.copy()
B1S000600_day_df = B1S000600_day_df.loc[B1S000600_day_df['METRIC_CD']==v_metric_cd]
B1S000600_day_df = B1S000600_day_df.loc[B1S000600_day_df['TM_KEY_DAY']>=v_tm_key_day]
# B1S000600_day_df = B1S000600_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

B1S000600_day_df['P'] = np.where(B1S000600_day_df['AREA_TYPE']=='P', B1S000600_day_df['ACTUAL_SNAP'], 0)
B1S000600_day_df['G'] = np.where(B1S000600_day_df['AREA_TYPE']=='G', B1S000600_day_df['ACTUAL_SNAP'], 0)

B1S000600_day_df = B1S000600_day_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'P':'sum', 'G':'sum'})
B1S000600_day_df['Diff (P - G)'] = B1S000600_day_df['P'] - B1S000600_day_df['G']
B1S000600_day_df = B1S000600_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index()

mod_col_list = B1S000600_day_df.iloc[:, 5:8].columns.tolist()
for col in mod_col_list:
    B1S000600_day_df[col] = B1S000600_day_df[col].apply(lambda x: format(x, ',.0f'))
    
B1S000600_day_df

,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,PPN_TM,P,G,Diff (P - G)
0,202505,20250501,B1S000600,Prepaid Active Caller 30D Rolling,2025-05-22 23:32:26,"26,568,494","26,547,484","21,010"
1,202505,20250502,B1S000600,Prepaid Active Caller 30D Rolling,2025-05-22 23:32:26,"26,649,375","26,628,655","20,720"
2,202505,20250503,B1S000600,Prepaid Active Caller 30D Rolling,2025-05-22 23:32:26,"26,640,242","26,619,634","20,608"
3,202505,20250504,B1S000600,Prepaid Active Caller 30D Rolling,2025-05-22 23:32:26,"26,621,620","26,601,369","20,251"
4,202505,20250505,B1S000600,Prepaid Active Caller 30D Rolling,2025-05-22 23:32:26,"26,604,806","16,762,556","9,842,250"
5,202505,20250506,B1S000600,Prepaid Active Caller 30D Rolling,2025-05-22 23:32:26,"26,604,015","26,584,118","19,897"
6,202505,20250507,B1S000600,Prepaid Active Caller 30D Rolling,2025-05-22 23:32:26,"26,587,879","26,568,125","19,754"
7,202505,20250508,B1S000600,Prepaid Active Caller 30D Rolling,2025-05-22 23:32:26,"26,568,787","16,742,069","9,826,718"
8,202505,20250509,B1S000600,Prepaid Active Caller 30D Rolling,2025-05-22 23:32:26,"26,608,313","26,587,851","20,462"
9,202505,20250510,B1S000600,Prepaid Active Caller 30D Rolling,2025-05-22 23:32:26,"26,612,607","26,591,233","21,374"


In [42]:
''' TB1S000600 : Prepaid Active Caller 30D Rolling : TMH '''

v_metric_cd = 'TB1S000600'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB1S000600_day_df = chk_src_df.copy()
TB1S000600_day_df = TB1S000600_day_df.loc[TB1S000600_day_df['METRIC_CD']==v_metric_cd]
TB1S000600_day_df = TB1S000600_day_df.loc[TB1S000600_day_df['TM_KEY_DAY']>=v_tm_key_day]
# TB1S000600_day_df = TB1S000600_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

TB1S000600_day_df['P'] = np.where(TB1S000600_day_df['AREA_TYPE']=='P', TB1S000600_day_df['ACTUAL_SNAP'], 0)
TB1S000600_day_df['G'] = np.where(TB1S000600_day_df['AREA_TYPE']=='G', TB1S000600_day_df['ACTUAL_SNAP'], 0)

TB1S000600_day_df = TB1S000600_day_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'P':'sum', 'G':'sum'})
TB1S000600_day_df['Diff (P - G)'] = TB1S000600_day_df['P'] - TB1S000600_day_df['G']
TB1S000600_day_df = TB1S000600_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index()

mod_col_list = TB1S000600_day_df.iloc[:, 5:8].columns.tolist()
for col in mod_col_list:
    TB1S000600_day_df[col] = TB1S000600_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB1S000600_day_df

,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,PPN_TM,P,G,Diff (P - G)
0,202505,20250501,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-05-22 23:32:26,"16,730,396","16,710,970","19,426"
1,202505,20250502,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-05-22 23:32:26,"16,798,106","16,778,974","19,132"
2,202505,20250503,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-05-22 23:32:26,"16,796,420","16,777,374","19,046"
3,202505,20250504,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-05-22 23:32:26,"16,788,470","16,769,647","18,823"
4,202505,20250505,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-05-22 23:32:26,"16,781,086","16,762,556","18,530"
5,202505,20250506,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-05-22 23:32:26,"16,775,944","16,757,635","18,309"
6,202505,20250507,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-05-22 23:32:26,"16,768,764","16,750,670","18,094"
7,202505,20250508,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-05-22 23:32:26,"16,760,144","16,742,069","18,075"
8,202505,20250509,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-05-22 23:32:26,"16,785,995","16,767,226","18,769"
9,202505,20250510,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-05-22 23:32:26,"16,783,795","16,764,123","19,672"


In [43]:
''' DB1S000600 : Prepaid Active Caller 30D Rolling : DTAC '''

v_metric_cd = 'DB1S000600'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

DB1S000600_day_df = chk_src_df.copy()
DB1S000600_day_df = DB1S000600_day_df.loc[DB1S000600_day_df['METRIC_CD']==v_metric_cd]
DB1S000600_day_df = DB1S000600_day_df.loc[DB1S000600_day_df['TM_KEY_DAY']>=v_tm_key_day]
# DB1S000600_day_df = DB1S000600_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

DB1S000600_day_df['P'] = np.where(DB1S000600_day_df['AREA_TYPE']=='P', DB1S000600_day_df['ACTUAL_SNAP'], 0)
DB1S000600_day_df['G'] = np.where(DB1S000600_day_df['AREA_TYPE']=='G', DB1S000600_day_df['ACTUAL_SNAP'], 0)

DB1S000600_day_df = DB1S000600_day_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'P':'sum', 'G':'sum'})
DB1S000600_day_df['Diff (P - G)'] = DB1S000600_day_df['P'] - DB1S000600_day_df['G']
DB1S000600_day_df = DB1S000600_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index()

mod_col_list = DB1S000600_day_df.iloc[:, 5:8].columns.tolist()
for col in mod_col_list:
    DB1S000600_day_df[col] = DB1S000600_day_df[col].apply(lambda x: format(x, ',.0f'))
    
DB1S000600_day_df

,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,PPN_TM,P,G,Diff (P - G)
0,202505,20250501,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-05-22 23:32:26,"9,838,098","9,836,514","1,584"
1,202505,20250502,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-05-22 23:32:26,"9,851,269","9,849,681","1,588"
2,202505,20250503,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-05-22 23:32:26,"9,843,822","9,842,260","1,562"
3,202505,20250504,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-05-22 23:32:26,"9,833,150","9,831,722","1,428"
4,202505,20250505,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-05-22 23:32:26,"9,823,720",0,"9,823,720"
5,202505,20250506,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-05-22 23:32:26,"9,828,071","9,826,483","1,588"
6,202505,20250507,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-05-22 23:32:26,"9,819,115","9,817,455","1,660"
7,202505,20250508,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-05-22 23:32:26,"9,808,643",0,"9,808,643"
8,202505,20250509,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-05-22 23:32:26,"9,822,318","9,820,625","1,693"
9,202505,20250510,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-05-22 23:32:26,"9,828,812","9,827,110","1,702"


### Area 080 : DTAC --> Done !!!

In [2]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME
        , SUM(CASE WHEN AREA_CD = '080' THEN ACTUAL_SNAP END) ACTUAL_080
        , SUM(CASE WHEN AREA_CD = '080' THEN TARGET_SNAP END) TARGET_080
        , SUM(CASE WHEN AREA_CD = '016' THEN ACTUAL_SNAP END) ACTUAL_016
        , SUM(CASE WHEN AREA_CD = '016' THEN TARGET_SNAP END) TARGET_016
        , SUM(CASE WHEN AREA_CD = '040' THEN ACTUAL_SNAP END) ACTUAL_040
        , SUM(CASE WHEN AREA_CD = '040' THEN TARGET_SNAP END) TARGET_040
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF, MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = 'DB1R000100' --Prepaid Revenue : DTAC
    --WHERE METRIC_CD = 'DB2R000100' --Postpaid Revenue : DTAC
    --WHERE METRIC_CD = 'DB2R010100' --Postpaid Revenue B2C : DTAC
    --WHERE METRIC_CD = 'DB2R020100' --Postpaid Revenue B2B : DTAC
    AND AREA_CD IN ('016', '040', '080')
    AND TM_KEY_DAY >= {v_start_date} 
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME
    ORDER BY TM_KEY_DAY, METRIC_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    # txn_df = chk_src_df.copy()
    # txn_df = txn_df.replace(np.nan, 0)
    # mod_col_list = txn_df.iloc[:, 4:10].columns.tolist()
    # for col in mod_col_list:
    #     txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    # print(f'\n{txn_df.to_string(max_cols=100)}')
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-22, 09:51:44
   -> v_start_date: 20250101


In [4]:
''' Sample Data '''

txn_df = chk_src_df.copy()
txn_df = txn_df.replace(np.nan, 0)
mod_col_list = txn_df.iloc[:, 4:10].columns.tolist()
for col in mod_col_list:
    txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{txn_df.to_string(max_cols=100)}')
txn_df

C:\Users\Narut4\AppData\Local\Temp\ipykernel_24672\3747703875.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  txn_df = txn_df.replace(np.nan, 0)


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,ACTUAL_080,TARGET_080,ACTUAL_016,TARGET_016,ACTUAL_040,TARGET_040,ACTUAL_AS_OF,PPN_TM
0,202501,20250101,DB1R000100,Prepaid Revenue : DTAC,"1,458,637",0,0,0,0,0,20250101,2025-05-21 05:46:58
1,202501,20250102,DB1R000100,Prepaid Revenue : DTAC,"1,378,370",0,0,0,0,0,20250102,2025-05-21 05:46:58
2,202501,20250103,DB1R000100,Prepaid Revenue : DTAC,"1,330,136",0,0,0,0,0,20250103,2025-05-21 05:46:58
3,202501,20250104,DB1R000100,Prepaid Revenue : DTAC,"1,229,670",0,0,0,0,0,20250104,2025-05-21 05:46:58
4,202501,20250105,DB1R000100,Prepaid Revenue : DTAC,"1,312,542",0,0,0,0,0,20250105,2025-05-21 05:46:58
...,...,...,...,...,...,...,...,...,...,...,...,...
135,202505,20250516,DB1R000100,Prepaid Revenue : DTAC,"1,234,064","1,221,957",0,0,0,0,20250516,2025-05-21 05:46:58
136,202505,20250517,DB1R000100,Prepaid Revenue : DTAC,"1,205,304","1,205,382",0,0,0,0,20250517,2025-05-21 05:46:58
137,202505,20250518,DB1R000100,Prepaid Revenue : DTAC,"1,295,234","1,162,579",0,0,0,0,20250518,2025-05-21 05:46:58
138,202505,20250519,DB1R000100,Prepaid Revenue : DTAC,"1,227,031","1,160,581",0,0,0,0,20250519,2025-05-21 05:46:58


In [3]:
jan25_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202501].copy()
jan25_df = jan25_df.replace(np.nan, 0)
mod_col_list = jan25_df.iloc[:, 4:10].columns.tolist()
for col in mod_col_list:
    jan25_df[col] = jan25_df[col].apply(lambda x: format(x, ',.0f'))
    
jan25_df

C:\Users\Narut4\AppData\Local\Temp\ipykernel_18160\1336843066.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  jan25_df = jan25_df.replace(np.nan, 0)


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,ACTUAL_080,TARGET_080,ACTUAL_016,TARGET_016,ACTUAL_040,TARGET_040,ACTUAL_AS_OF,PPN_TM
0,202501,20250101,DB1R000100,Prepaid Revenue : DTAC,"1,458,637","1,425,288",0,0,0,0,20250101,2025-05-22 00:56:22
1,202501,20250102,DB1R000100,Prepaid Revenue : DTAC,"1,378,370","1,457,336",0,0,0,0,20250102,2025-05-22 00:56:22
2,202501,20250103,DB1R000100,Prepaid Revenue : DTAC,"1,330,136","1,413,942",0,0,0,0,20250103,2025-05-22 00:56:22
3,202501,20250104,DB1R000100,Prepaid Revenue : DTAC,"1,229,670","1,358,850",0,0,0,0,20250104,2025-05-22 00:56:22
4,202501,20250105,DB1R000100,Prepaid Revenue : DTAC,"1,312,542","1,362,544",0,0,0,0,20250105,2025-05-22 00:56:22
5,202501,20250106,DB1R000100,Prepaid Revenue : DTAC,"1,257,104","1,290,379",0,0,0,0,20250106,2025-05-22 00:56:22
6,202501,20250107,DB1R000100,Prepaid Revenue : DTAC,"1,264,117","1,248,284",0,0,0,0,20250107,2025-05-22 00:56:22
7,202501,20250108,DB1R000100,Prepaid Revenue : DTAC,"1,205,018","1,265,016",0,0,0,0,20250108,2025-05-22 00:56:22
8,202501,20250109,DB1R000100,Prepaid Revenue : DTAC,"1,075,793","1,219,491",0,0,0,0,20250109,2025-05-22 00:56:22
9,202501,20250110,DB1R000100,Prepaid Revenue : DTAC,"1,146,732","1,246,177",0,0,0,0,20250110,2025-05-22 00:56:22


In [4]:
feb25_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202502].copy()
feb25_df = feb25_df.replace(np.nan, 0)
mod_col_list = feb25_df.iloc[:, 4:10].columns.tolist()
for col in mod_col_list:
    feb25_df[col] = feb25_df[col].apply(lambda x: format(x, ',.0f'))
    
feb25_df

C:\Users\Narut4\AppData\Local\Temp\ipykernel_18160\4083946966.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  feb25_df = feb25_df.replace(np.nan, 0)


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,ACTUAL_080,TARGET_080,ACTUAL_016,TARGET_016,ACTUAL_040,TARGET_040,ACTUAL_AS_OF,PPN_TM
31,202502,20250201,DB1R000100,Prepaid Revenue : DTAC,"1,537,712","1,457,974",0,0,0,0,20250201,2025-05-22 00:56:22
32,202502,20250202,DB1R000100,Prepaid Revenue : DTAC,"1,472,950","1,395,283",0,0,0,0,20250202,2025-05-22 00:56:22
33,202502,20250203,DB1R000100,Prepaid Revenue : DTAC,"1,361,083","1,376,123",0,0,0,0,20250203,2025-05-22 00:56:22
34,202502,20250204,DB1R000100,Prepaid Revenue : DTAC,"1,271,479","1,314,550",0,0,0,0,20250204,2025-05-22 00:56:22
35,202502,20250205,DB1R000100,Prepaid Revenue : DTAC,"1,291,841","1,309,221",0,0,0,0,20250205,2025-05-22 00:56:22
36,202502,20250206,DB1R000100,Prepaid Revenue : DTAC,"1,338,937","1,304,648",0,0,0,0,20250206,2025-05-22 00:56:22
37,202502,20250207,DB1R000100,Prepaid Revenue : DTAC,"1,230,067","1,273,728",0,0,0,0,20250207,2025-05-22 00:56:22
38,202502,20250208,DB1R000100,Prepaid Revenue : DTAC,"1,200,707","1,278,664",0,0,0,0,20250208,2025-05-22 00:56:22
39,202502,20250209,DB1R000100,Prepaid Revenue : DTAC,"1,169,782","1,214,903",0,0,0,0,20250209,2025-05-22 00:56:22
40,202502,20250210,DB1R000100,Prepaid Revenue : DTAC,"1,130,583","1,267,316",0,0,0,0,20250210,2025-05-22 00:56:22


In [5]:
mar25_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202503].copy()
mar25_df = mar25_df.replace(np.nan, 0)
mod_col_list = mar25_df.iloc[:, 4:10].columns.tolist()
for col in mod_col_list:
    mar25_df[col] = mar25_df[col].apply(lambda x: format(x, ',.0f'))
    
mar25_df

C:\Users\Narut4\AppData\Local\Temp\ipykernel_18160\3737428264.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mar25_df = mar25_df.replace(np.nan, 0)


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,ACTUAL_080,TARGET_080,ACTUAL_016,TARGET_016,ACTUAL_040,TARGET_040,ACTUAL_AS_OF,PPN_TM
59,202503,20250301,DB1R000100,Prepaid Revenue : DTAC,"1,506,675","1,567,283",0,0,0,0,20250301,2025-05-22 00:56:22
60,202503,20250302,DB1R000100,Prepaid Revenue : DTAC,"1,546,780","1,476,098",0,0,0,0,20250302,2025-05-22 00:56:22
61,202503,20250303,DB1R000100,Prepaid Revenue : DTAC,"1,449,951","1,384,844",0,0,0,0,20250303,2025-05-22 00:56:22
62,202503,20250304,DB1R000100,Prepaid Revenue : DTAC,"1,348,670","1,327,416",0,0,0,0,20250304,2025-05-22 00:56:22
63,202503,20250305,DB1R000100,Prepaid Revenue : DTAC,"1,342,948","1,359,366",0,0,0,0,20250305,2025-05-22 00:56:22
64,202503,20250306,DB1R000100,Prepaid Revenue : DTAC,"1,279,234","1,349,882",0,0,0,0,20250306,2025-05-22 00:56:22
65,202503,20250307,DB1R000100,Prepaid Revenue : DTAC,"1,247,824","1,283,626",0,0,0,0,20250307,2025-05-22 00:56:22
66,202503,20250308,DB1R000100,Prepaid Revenue : DTAC,"1,248,182","1,238,708",0,0,0,0,20250308,2025-05-22 00:56:22
67,202503,20250309,DB1R000100,Prepaid Revenue : DTAC,"1,225,405","1,167,094",0,0,0,0,20250309,2025-05-22 00:56:22
68,202503,20250310,DB1R000100,Prepaid Revenue : DTAC,"1,149,640","1,152,327",0,0,0,0,20250310,2025-05-22 00:56:22


In [6]:
apr25_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202504].copy()
apr25_df = apr25_df.replace(np.nan, 0)
mod_col_list = apr25_df.iloc[:, 4:10].columns.tolist()
for col in mod_col_list:
    apr25_df[col] = apr25_df[col].apply(lambda x: format(x, ',.0f'))
    
apr25_df

C:\Users\Narut4\AppData\Local\Temp\ipykernel_18160\1557403921.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  apr25_df = apr25_df.replace(np.nan, 0)


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,ACTUAL_080,TARGET_080,ACTUAL_016,TARGET_016,ACTUAL_040,TARGET_040,ACTUAL_AS_OF,PPN_TM
90,202504,20250401,DB1R000100,Prepaid Revenue : DTAC,"1,575,177","1,339,649",0,0,0,0,20250401,2025-05-22 00:56:22
91,202504,20250402,DB1R000100,Prepaid Revenue : DTAC,"1,427,780","1,434,931",0,0,0,0,20250402,2025-05-22 00:56:22
92,202504,20250403,DB1R000100,Prepaid Revenue : DTAC,"1,463,032","1,298,233",0,0,0,0,20250403,2025-05-22 00:56:22
93,202504,20250404,DB1R000100,Prepaid Revenue : DTAC,"1,330,197","1,248,136",0,0,0,0,20250404,2025-05-22 00:56:22
94,202504,20250405,DB1R000100,Prepaid Revenue : DTAC,"1,326,432","1,332,687",0,0,0,0,20250405,2025-05-22 00:56:22
95,202504,20250406,DB1R000100,Prepaid Revenue : DTAC,"1,295,827","1,262,578",0,0,0,0,20250406,2025-05-22 00:56:22
96,202504,20250407,DB1R000100,Prepaid Revenue : DTAC,"1,197,326","1,217,703",0,0,0,0,20250407,2025-05-22 00:56:22
97,202504,20250408,DB1R000100,Prepaid Revenue : DTAC,"1,190,268","1,176,555",0,0,0,0,20250408,2025-05-22 00:56:22
98,202504,20250409,DB1R000100,Prepaid Revenue : DTAC,"1,138,984","1,159,320",0,0,0,0,20250409,2025-05-22 00:56:22
99,202504,20250410,DB1R000100,Prepaid Revenue : DTAC,"1,157,204","1,224,347",0,0,0,0,20250410,2025-05-22 00:56:22


In [7]:
may25_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202505].copy()
may25_df = may25_df.replace(np.nan, 0)
mod_col_list = may25_df.iloc[:, 4:10].columns.tolist()
for col in mod_col_list:
    may25_df[col] = may25_df[col].apply(lambda x: format(x, ',.0f'))
    
may25_df

C:\Users\Narut4\AppData\Local\Temp\ipykernel_18160\4129942621.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  may25_df = may25_df.replace(np.nan, 0)


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,ACTUAL_080,TARGET_080,ACTUAL_016,TARGET_016,ACTUAL_040,TARGET_040,ACTUAL_AS_OF,PPN_TM
120,202505,20250501,DB1R000100,Prepaid Revenue : DTAC,"1,635,728","1,536,962",0,0,0,0,20250501,2025-05-22 00:56:22
121,202505,20250502,DB1R000100,Prepaid Revenue : DTAC,"1,530,001","1,416,205",0,0,0,0,20250502,2025-05-22 00:56:22
122,202505,20250503,DB1R000100,Prepaid Revenue : DTAC,"1,401,946","1,356,224",0,0,0,0,20250503,2025-05-22 00:56:22
123,202505,20250504,DB1R000100,Prepaid Revenue : DTAC,"1,315,014","1,272,045",0,0,0,0,20250504,2025-05-22 00:56:22
124,202505,20250505,DB1R000100,Prepaid Revenue : DTAC,"1,314,119","1,312,972",0,0,0,0,20250505,2025-05-22 00:56:22
125,202505,20250506,DB1R000100,Prepaid Revenue : DTAC,"1,271,558","1,248,316",0,0,0,0,20250506,2025-05-22 00:56:22
126,202505,20250507,DB1R000100,Prepaid Revenue : DTAC,"1,283,653","1,241,571",0,0,0,0,20250507,2025-05-22 00:56:22
127,202505,20250508,DB1R000100,Prepaid Revenue : DTAC,"1,288,672","1,224,601",0,0,0,0,20250508,2025-05-22 00:56:22
128,202505,20250509,DB1R000100,Prepaid Revenue : DTAC,"1,172,239","1,180,005",0,0,0,0,20250509,2025-05-22 00:56:22
129,202505,20250510,DB1R000100,Prepaid Revenue : DTAC,"1,162,885","1,204,041",0,0,0,0,20250510,2025-05-22 00:56:22


### Prepaid Revenue : TMH --> Done !!!

In [37]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250401
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY --, CENTER, PRODUCT_GRP, METRIC_GRP
        , METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD
        , AREA_NAME, ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH --, TARGET_SNAP
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = 'TB1R000100' --Prepaid Revenue : TMH
    AND AREA_CD = 'P'
    AND TM_KEY_DAY >= {v_start_date} 
    ORDER BY TM_KEY_DAY, CENTER, PRODUCT_GRP, METRIC_GRP, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    txn_df = chk_src_df.copy()
    # txn_df = txn_df.replace(np.nan, 0)
    mod_col_list = txn_df.iloc[:, 6:8].columns.tolist()
    for col in mod_col_list:
        txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{txn_df.to_string(max_cols=10)}') #max_rows=1000
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-23, 14:06:05
   -> v_start_date: 20250401

    TM_KEY_MTH  TM_KEY_DAY   METRIC_CD            METRIC_NAME   AREA_NAME  ACTUAL_AS_OF  ACTUAL_SNAP ACTUAL_AGG_MTH              PPN_TM
0       202504    20250401  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250401  101,504,221    101,504,221 2025-05-22 23:32:26
1       202504    20250402  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250402   92,119,156    193,623,377 2025-05-22 23:32:26
2       202504    20250403  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250403   85,407,783    279,031,159 2025-05-22 23:32:26
3       202504    20250404  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250404   85,737,245    364,768,404 2025-05-22 23:32:26
4       202504    20250405  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250405   84,177,907    448,946,311 2025-05-22 23:32:26
5       202504    20250406  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250406   80,74

### Monthly Revenue ARPU --> Monitoring...

In [14]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY --, CENTER, PRODUCT_GRP, METRIC_GRP
        , METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD
        , AREA_NAME, ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH --, TARGET_SNAP
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B7R000102' --Prepaid ARPU
        , 'DB7R000102' --Prepaid ARPU : DTAC
        , 'TB7R000102' --Prepaid ARPU : TMH
        , 'B7R000101' --Postpaid ARPU
        , 'B7R010101' --Postpaid ARPU B2C
        , 'DB7R000101' --Postpaid ARPU : DTAC
        , 'DB7R010101' --Postpaid ARPU B2C : DTAC
        , 'TB7R000101' --Postpaid ARPU : TMH
        , 'TB7R010101' --Postpaid ARPU B2C : TMH
        , 'TB7R000103' --TOL ARPU
        , 'TB7R000104' --TVS ARPU
        )
    AND AREA_CD = 'P'
    AND TM_KEY_DAY >= {v_start_date} 
    --ORDER BY TM_KEY_DAY, CENTER, PRODUCT_GRP, METRIC_GRP, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # # Display
    # txn_df = chk_src_df.copy()
    # mod_col_list = txn_df.iloc[:, 6:8].columns.tolist()
    # for col in mod_col_list:
    #     txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    # print(f'\n{txn_df.to_string(max_cols=10)}') #max_rows=1000

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-02, 10:50:51
   -> v_start_date: 20250101


In [15]:
''' B7R000102 : Prepaid ARPU '''

v_metric_cd = 'B7R000102'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nB7R000102 : Prepaid ARPU...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B7R000102_day_df = chk_src_df.copy()
B7R000102_day_df = B7R000102_day_df.loc[B7R000102_day_df['METRIC_CD']==v_metric_cd]
# B7R000102_day_df = B7R000102_day_df.loc[B7R000102_day_df['TM_KEY_DAY']>=v_tm_key_day]
B7R000102_day_df = B7R000102_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = B7R000102_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    B7R000102_day_df[col] = B7R000102_day_df[col].apply(lambda x: format(x, ',.1f'))
    
B7R000102_day_df


B7R000102 : Prepaid ARPU...

   -> v_metric_cd: B7R000102


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,B7R000102,Prepaid ARPU,Nationwide,20250101,172.9,172.9,2025-05-02 08:51:18
1,202501,20250102,B7R000102,Prepaid ARPU,Nationwide,20250102,172.8,172.8,2025-05-02 08:51:18
2,202501,20250103,B7R000102,Prepaid ARPU,Nationwide,20250103,173.2,173.2,2025-05-02 08:51:18
3,202501,20250104,B7R000102,Prepaid ARPU,Nationwide,20250104,173.4,173.4,2025-05-02 08:51:18
4,202501,20250105,B7R000102,Prepaid ARPU,Nationwide,20250105,173.5,173.5,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,B7R000102,Prepaid ARPU,Nationwide,20250427,151.5,151.5,2025-05-02 08:51:18
117,202504,20250428,B7R000102,Prepaid ARPU,Nationwide,20250428,151.5,151.5,2025-05-02 08:51:18
118,202504,20250429,B7R000102,Prepaid ARPU,Nationwide,20250428,0.0,151.5,2025-05-02 08:51:18
119,202504,20250430,B7R000102,Prepaid ARPU,Nationwide,20250428,0.0,151.5,2025-05-02 08:51:18


In [16]:
''' B7R010101 : Postpaid ARPU B2C '''

v_metric_cd = 'B7R010101'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nB7R010101 : Postpaid ARPU B2C...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B7R010101_day_df = chk_src_df.copy()
B7R010101_day_df = B7R010101_day_df.loc[B7R010101_day_df['METRIC_CD']==v_metric_cd]
# B7R010101_day_df = B7R010101_day_df.loc[B7R010101_day_df['TM_KEY_DAY']>=v_tm_key_day]
B7R010101_day_df = B7R010101_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = B7R010101_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    B7R010101_day_df[col] = B7R010101_day_df[col].apply(lambda x: format(x, ',.1f'))
    
B7R010101_day_df


B7R010101 : Postpaid ARPU B2C...

   -> v_metric_cd: B7R010101


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,B7R010101,Postpaid ARPU B2C,Nationwide,20250101,416.8,416.8,2025-05-02 08:51:18
1,202501,20250102,B7R010101,Postpaid ARPU B2C,Nationwide,20250102,417.0,417.0,2025-05-02 08:51:18
2,202501,20250103,B7R010101,Postpaid ARPU B2C,Nationwide,20250103,417.0,417.0,2025-05-02 08:51:18
3,202501,20250104,B7R010101,Postpaid ARPU B2C,Nationwide,20250104,417.0,417.0,2025-05-02 08:51:18
4,202501,20250105,B7R010101,Postpaid ARPU B2C,Nationwide,20250105,416.7,416.7,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,B7R010101,Postpaid ARPU B2C,Nationwide,20250427,416.7,416.7,2025-05-02 08:51:18
117,202504,20250428,B7R010101,Postpaid ARPU B2C,Nationwide,20250428,392.3,392.3,2025-05-02 08:51:18
118,202504,20250429,B7R010101,Postpaid ARPU B2C,Nationwide,20250428,0.0,392.3,2025-05-02 08:51:18
119,202504,20250430,B7R010101,Postpaid ARPU B2C,Nationwide,20250428,0.0,392.3,2025-05-02 08:51:18


In [17]:
''' TB7R010101 : Postpaid ARPU B2C : TMH '''

v_metric_cd = 'TB7R010101'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTB7R010101 : Postpaid ARPU B2C : TMH...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB7R010101_day_df = chk_src_df.copy()
TB7R010101_day_df = TB7R010101_day_df.loc[TB7R010101_day_df['METRIC_CD']==v_metric_cd]
# TB7R010101_day_df = TB7R010101_day_df.loc[TB7R010101_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB7R010101_day_df = TB7R010101_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = TB7R010101_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    TB7R010101_day_df[col] = TB7R010101_day_df[col].apply(lambda x: format(x, ',.1f'))
    
TB7R010101_day_df


TB7R010101 : Postpaid ARPU B2C : TMH...

   -> v_metric_cd: TB7R010101


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250101,371.6,371.6,2025-05-02 08:51:18
1,202501,20250102,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250102,372.0,372.0,2025-05-02 08:51:18
2,202501,20250103,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250103,372.0,372.0,2025-05-02 08:51:18
3,202501,20250104,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250104,372.0,372.0,2025-05-02 08:51:18
4,202501,20250105,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250105,371.9,371.9,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250427,371.7,371.7,2025-05-02 08:51:18
117,202504,20250428,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250428,333.7,333.7,2025-05-02 08:51:18
118,202504,20250429,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250429,333.9,333.9,2025-05-02 08:51:18
119,202504,20250430,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250429,0.0,333.9,2025-05-02 08:51:18


In [18]:
''' DB7R010101 : Postpaid ARPU B2C : DTAC '''

v_metric_cd = 'DB7R010101'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nDB7R010101 : Postpaid ARPU B2C : DTAC...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

DB7R010101_day_df = chk_src_df.copy()
DB7R010101_day_df = DB7R010101_day_df.loc[DB7R010101_day_df['METRIC_CD']==v_metric_cd]
# DB7R010101_day_df = DB7R010101_day_df.loc[DB7R010101_day_df['TM_KEY_DAY']>=v_tm_key_day]
DB7R010101_day_df = DB7R010101_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = DB7R010101_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    DB7R010101_day_df[col] = DB7R010101_day_df[col].apply(lambda x: format(x, ',.1f'))
    
DB7R010101_day_df


DB7R010101 : Postpaid ARPU B2C : DTAC...

   -> v_metric_cd: DB7R010101


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250101,494.7,494.7,2025-05-02 08:51:18
1,202501,20250102,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250102,494.5,494.5,2025-05-02 08:51:18
2,202501,20250103,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250103,494.6,494.6,2025-05-02 08:51:18
3,202501,20250104,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250104,494.4,494.4,2025-05-02 08:51:18
4,202501,20250105,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250105,494.1,494.1,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250427,496.4,496.4,2025-05-02 08:51:18
117,202504,20250428,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250428,496.2,496.2,2025-05-02 08:51:18
118,202504,20250429,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250428,0.0,496.2,2025-05-02 08:51:18
119,202504,20250430,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250428,0.0,496.2,2025-05-02 08:51:18


In [19]:
''' TB7R000103 : TOL ARPU '''

v_metric_cd = 'TB7R000103'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTB7R000103 : TOL ARPU...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB7R000103_day_df = chk_src_df.copy()
TB7R000103_day_df = TB7R000103_day_df.loc[TB7R000103_day_df['METRIC_CD']==v_metric_cd]
# TB7R000103_day_df = TB7R000103_day_df.loc[TB7R000103_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB7R000103_day_df = TB7R000103_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = TB7R000103_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    TB7R000103_day_df[col] = TB7R000103_day_df[col].apply(lambda x: format(x, ',.1f'))
    
TB7R000103_day_df


TB7R000103 : TOL ARPU...

   -> v_metric_cd: TB7R000103


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,TB7R000103,TOL ARPU,Nationwide,20250101,510.9,510.9,2025-05-02 08:51:18
1,202501,20250102,TB7R000103,TOL ARPU,Nationwide,20250102,510.8,510.8,2025-05-02 08:51:18
2,202501,20250103,TB7R000103,TOL ARPU,Nationwide,20250103,510.9,510.9,2025-05-02 08:51:18
3,202501,20250104,TB7R000103,TOL ARPU,Nationwide,20250104,510.8,510.8,2025-05-02 08:51:18
4,202501,20250105,TB7R000103,TOL ARPU,Nationwide,20250105,510.6,510.6,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,TB7R000103,TOL ARPU,Nationwide,20250427,514.3,514.3,2025-05-02 08:51:18
117,202504,20250428,TB7R000103,TOL ARPU,Nationwide,20250428,513.3,513.3,2025-05-02 08:51:18
118,202504,20250429,TB7R000103,TOL ARPU,Nationwide,20250428,0.0,513.3,2025-05-02 08:51:18
119,202504,20250430,TB7R000103,TOL ARPU,Nationwide,20250428,0.0,513.3,2025-05-02 08:51:18


In [20]:
''' TB7R000104 : TVS ARPU '''

v_metric_cd = 'TB7R000104'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTB7R000104 : Postpaid ARPU B2C...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB7R000104_day_df = chk_src_df.copy()
TB7R000104_day_df = TB7R000104_day_df.loc[TB7R000104_day_df['METRIC_CD']==v_metric_cd]
# TB7R000104_day_df = TB7R000104_day_df.loc[TB7R000104_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB7R000104_day_df = TB7R000104_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = TB7R000104_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    TB7R000104_day_df[col] = TB7R000104_day_df[col].apply(lambda x: format(x, ',.1f'))
    
TB7R000104_day_df


TB7R000104 : Postpaid ARPU B2C...

   -> v_metric_cd: TB7R000104


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,TB7R000104,TVS ARPU,Nationwide,20250101,260.4,260.4,2025-05-02 08:51:18
1,202501,20250102,TB7R000104,TVS ARPU,Nationwide,20250102,259.8,259.8,2025-05-02 08:51:18
2,202501,20250103,TB7R000104,TVS ARPU,Nationwide,20250103,259.8,259.8,2025-05-02 08:51:18
3,202501,20250104,TB7R000104,TVS ARPU,Nationwide,20250104,259.9,259.9,2025-05-02 08:51:18
4,202501,20250105,TB7R000104,TVS ARPU,Nationwide,20250105,259.9,259.9,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,TB7R000104,TVS ARPU,Nationwide,20250427,259.0,259.0,2025-05-02 08:51:18
117,202504,20250428,TB7R000104,TVS ARPU,Nationwide,20250428,258.0,258.0,2025-05-02 08:51:18
118,202504,20250429,TB7R000104,TVS ARPU,Nationwide,20250428,0.0,258.0,2025-05-02 08:51:18
119,202504,20250430,TB7R000104,TVS ARPU,Nationwide,20250428,0.0,258.0,2025-05-02 08:51:18


### * Revenue Corporate (C, P) --> Pending...

In [12]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100' --Total Revenue
        , 'DB0R000100' --Total Revenue : DTAC
        , 'TB0R000100' --Total Revenue : TRUE
        , 'B0R000101' --Mobile Revenue
        , 'DB0R000101' --Mobile Revenue : DTAC
        , 'TB0R000101' --Mobile Revenue : TMH
        , 'B2R000100' --Postpaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'DB2R000100' --Postpaid Revenue : DTAC
        , 'DB2R010100' --Postpaid Revenue B2C : DTAC
        , 'TB2R000100' --Postpaid Revenue : TMH
        , 'TB2R010100' --Postpaid Revenue B2C : TMH
        , 'B1R000100' --Prepaid Revenue
        , 'DB1R000100' --Prepaid Revenue : DTAC
        , 'TB1R000100' --Prepaid Revenue : TMH
        , 'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-26, 14:44:44
   -> v_start_date: 20250101


#### Sample Data

In [13]:
sample_mth_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202501].copy()

sample_mth_df['METRIC_TMP'] = sample_mth_df['METRIC_NAME'].replace(r' : DTAC| : TRUE| : TMH', '', regex=True)
sample_mth_df['ALL'] = np.where(sample_mth_df['COMP_CD']=='ALL', sample_mth_df['C'], 0)
sample_mth_df['TRUE'] = np.where(sample_mth_df['COMP_CD']=='TRUE', sample_mth_df['C'], 0)
sample_mth_df['DTAC'] = np.where(sample_mth_df['COMP_CD']=='DTAC', sample_mth_df['C'], 0)

sample_mth_df = sample_mth_df.groupby(['TM_KEY_MTH', 'METRIC_TMP']).agg({'ACTUAL_AS_OF':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum', 'PPN_TM':'max'})
sample_mth_df = sample_mth_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_TMP']).reset_index()

mod_col_list = sample_mth_df.iloc[:, 3:6].columns.tolist()
for col in mod_col_list:
    sample_mth_df[col] = sample_mth_df[col].apply(lambda x: format(x, ',.0f'))

sample_mth_df

,TM_KEY_MTH,METRIC_TMP,ACTUAL_AS_OF,ALL,TRUE,DTAC,PPN_TM
0,202501,Mobile Revenue,20250131.0,"9,784,556,756","2,444,926,240","7,339,630,516",2025-05-26 00:09:03
1,202501,Postpaid Revenue,20250131.0,"6,470,994,119","3,731,079,534","2,739,914,585",2025-05-26 03:06:46
2,202501,Postpaid Revenue B2C,20250131.0,"5,868,396,046","3,336,770,490","2,531,625,556",2025-05-26 03:06:46
3,202501,Prepaid Revenue,20250131.0,"4,284,739,583","2,444,926,240","1,839,813,343",2025-05-26 03:06:46
4,202501,TOL Revenue,20250128.0,0,"1,597,056,733",0,2025-05-26 03:06:46
5,202501,TVS Revenue,20250131.0,0,"417,316,338",0,2025-05-26 03:06:46
6,202501,Total Revenue,20250131.0,"11,798,929,827","4,459,299,311","7,339,630,516",2025-05-26 03:06:46


In [14]:
sample_day_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==20250102].copy()

mod_col_list = sample_day_df.iloc[:, 7:12].columns.tolist()
for col in mod_col_list:
    sample_day_df[col] = sample_day_df[col].apply(lambda x: format(x, ',.0f'))

sample_day_df = sample_day_df.sort_values(by=['TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD']).reset_index(drop=True)
sample_day_df

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,ACTUAL_AS_OF,C,P,G,H,HH,PPN_TM
0,202501,20250102,All Services,ALL,B0R000100,Total Revenue,20250102.0,"641,115,933","1,253,473,170",nan,nan,nan,2025-05-26 00:09:03
1,202501,20250102,All Services,DTAC,DB0R000100,Total Revenue : DTAC,20250102.0,"301,030,841","301,035,463","301,035,463","300,771,410","300,771,410",2025-05-26 03:06:46
2,202501,20250102,All Services,TRUE,TB0R000100,Total Revenue : TRUE,20250102.0,"340,085,092","952,437,707","335,844,364","335,769,394","335,767,827",2025-05-26 03:06:46
3,202501,20250102,Mobile,ALL,B0R000101,Mobile Revenue,20250102.0,"389,541,647","1,003,155,781",nan,nan,nan,2025-05-26 00:09:03
4,202501,20250102,Mobile,DTAC,DB0R000101,Mobile Revenue : DTAC,20250102.0,"301,030,841","301,035,463",nan,nan,nan,2025-05-26 00:09:03
5,202501,20250102,Mobile,TRUE,TB0R000101,Mobile Revenue : TMH,20250102.0,"88,510,806","702,120,319",nan,nan,nan,2025-05-26 00:09:03
6,202501,20250102,Postpaid,ALL,B2R000100,Postpaid Revenue,20250102.0,"847,437,062","847,437,062",nan,nan,nan,2025-05-26 00:09:03
7,202501,20250102,Postpaid,ALL,B2R010100,Postpaid Revenue B2C,20250102.0,"577,284,228","577,284,228","577,283,416","577,281,093","577,281,093",2025-05-26 03:06:46
8,202501,20250102,Postpaid,DTAC,DB2R000100,Postpaid Revenue : DTAC,20250102.0,"233,715,912","233,715,912","233,715,912","233,452,187","233,452,187",2025-05-26 03:06:46
9,202501,20250102,Postpaid,DTAC,DB2R010100,Postpaid Revenue B2C : DTAC,20250102.0,"97,527,120","97,527,120","97,527,120","97,526,880","97,526,880",2025-05-26 03:06:46


#### Product Summary

In [15]:
''' Total Revenue '''

v_total_metric_list = ['B0R000100', 'TB0R000100', 'DB0R000100']
# v_tm_key_mth = 202501
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTotal Revenue...\n')
print(f'   -> v_total_metric_list: {v_total_metric_list}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B0R000100_day_df = chk_src_df.copy()
B0R000100_day_df = B0R000100_day_df.loc[B0R000100_day_df['METRIC_CD'].isin(v_total_metric_list)]
# B0R000100_day_df = B0R000100_day_df.loc[B0R000100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# B0R000100_day_df = B0R000100_day_df.loc[B0R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]

# B0R000100_day_df['METRIC_TMP'] = B0R000100_day_df['METRIC_NAME'].replace(r' : DTAC| : TRUE| : TMH', '', regex=True)
# B0R000100_day_df['ALL'] = np.where(B0R000100_day_df['COMP_CD']=='ALL', B0R000100_day_df['ACTUAL_SNAP'], 0)
# B0R000100_day_df['TRUE'] = np.where(B0R000100_day_df['COMP_CD']=='TRUE', B0R000100_day_df['ACTUAL_SNAP'], 0)
# B0R000100_day_df['DTAC'] = np.where(B0R000100_day_df['COMP_CD']=='DTAC', B0R000100_day_df['ACTUAL_SNAP'], 0)

B0R000100_day_df = B0R000100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
B0R000100_day_df = B0R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
# B0R000100_day_df['ACTUAL_AS_OF'] = B0R000100_day_df['ACTUAL_AS_OF'].astype(str)

B0R000100_day_df['Diff (C - P)'] = B0R000100_day_df['C'] - B0R000100_day_df['P']
B0R000100_day_df = B0R000100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

mod_col_list = B0R000100_day_df.iloc[:, 4:8].columns.tolist()
for col in mod_col_list:
    B0R000100_day_df[col] = B0R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
B0R000100_day_df


Total Revenue...

   -> v_total_metric_list: ['B0R000100', 'TB0R000100', 'DB0R000100']


,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,Diff (C - P),C,P,G,PPN_TM
0,202501,All Services,B0R000100,Total Revenue,"-838,015,676","11,798,929,827","12,636,945,503",0,2025-05-26 00:09:03
1,202501,All Services,DB0R000100,Total Revenue : DTAC,"2,759,688,949","7,339,630,516","4,579,941,567","4,579,940,849",2025-05-26 03:06:46
2,202501,All Services,TB0R000100,Total Revenue : TRUE,"-3,597,704,625","4,459,299,311","8,057,003,936","4,316,603,800",2025-05-26 03:06:46
3,202502,All Services,B0R000100,Total Revenue,"-1,301,549,252","10,910,887,639","12,212,436,891",0,2025-05-26 00:09:03
4,202502,All Services,DB0R000100,Total Revenue : DTAC,"2,400,545,272","6,776,726,890","4,376,181,618","4,376,180,420",2025-05-26 03:06:46
5,202502,All Services,TB0R000100,Total Revenue : TRUE,"-3,702,094,524","4,134,160,749","7,836,255,273","4,067,708,891",2025-05-26 03:06:46
6,202503,All Services,B0R000100,Total Revenue,"-1,045,058,149","11,542,558,166","12,587,616,315",0,2025-05-26 00:09:03
7,202503,All Services,DB0R000100,Total Revenue : DTAC,"2,734,724,325","7,245,126,495","4,510,402,170","4,510,401,073",2025-05-26 03:06:46
8,202503,All Services,TB0R000100,Total Revenue : TRUE,"-3,779,782,474","4,297,431,671","8,077,214,145","4,307,216,697",2025-05-26 03:06:46
9,202504,All Services,B0R000100,Total Revenue,"-3,743,433,347","8,758,472,043","12,501,905,390",0,2025-05-26 00:09:03


In [16]:
''' Mobile Revenue '''

v_mobile_metric_list = ['B0R000101', 'TB0R000101', 'DB0R000101']
# v_tm_key_mth = 202501
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nMobile Revenue...\n')
print(f'   -> v_mobile_metric_list: {v_mobile_metric_list}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B0R000101_day_df = chk_src_df.copy()
B0R000101_day_df = B0R000101_day_df.loc[B0R000101_day_df['METRIC_CD'].isin(v_mobile_metric_list)]
# B0R000101_day_df = B0R000101_day_df.loc[B0R000101_day_df['TM_KEY_MTH']==v_tm_key_mth]
# B0R000101_day_df = B0R000101_day_df.loc[B0R000101_day_df['TM_KEY_DAY']>=v_tm_key_day]

B0R000101_day_df = B0R000101_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
B0R000101_day_df = B0R000101_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

B0R000101_day_df['Diff (C - P)'] = B0R000101_day_df['C'] - B0R000101_day_df['P']
B0R000101_day_df = B0R000101_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

mod_col_list = B0R000101_day_df.iloc[:, 4:8].columns.tolist()
for col in mod_col_list:
    B0R000101_day_df[col] = B0R000101_day_df[col].apply(lambda x: format(x, ',.0f'))
    
B0R000101_day_df


Mobile Revenue...

   -> v_mobile_metric_list: ['B0R000101', 'TB0R000101', 'DB0R000101']


,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,Diff (C - P),C,P,G,PPN_TM
0,202501,Mobile,B0R000101,Mobile Revenue,"-966,994,427","9,784,556,756","10,751,551,183",0,2025-05-26 00:09:03
1,202501,Mobile,DB0R000101,Mobile Revenue : DTAC,"2,759,688,949","7,339,630,516","4,579,941,567",0,2025-05-26 00:09:03
2,202501,Mobile,TB0R000101,Mobile Revenue : TMH,"-3,726,683,376","2,444,926,240","6,171,609,616",0,2025-05-26 00:09:03
3,202502,Mobile,B0R000101,Mobile Revenue,"-1,320,015,799","8,993,985,897","10,314,001,696",0,2025-05-26 00:09:03
4,202502,Mobile,DB0R000101,Mobile Revenue : DTAC,"2,400,545,272","6,776,726,890","4,376,181,618",0,2025-05-26 00:09:03
5,202502,Mobile,TB0R000101,Mobile Revenue : TMH,"-3,720,561,071","2,217,259,007","5,937,820,079",0,2025-05-26 00:09:03
6,202503,Mobile,B0R000101,Mobile Revenue,"-966,622,884","9,711,497,477","10,678,120,361",0,2025-05-26 00:09:03
7,202503,Mobile,DB0R000101,Mobile Revenue : DTAC,"2,734,724,325","7,245,126,495","4,510,402,170",0,2025-05-26 00:09:03
8,202503,Mobile,TB0R000101,Mobile Revenue : TMH,"-3,701,347,210","2,466,370,982","6,167,718,192",0,2025-05-26 00:09:03
9,202504,Mobile,B0R000101,Mobile Revenue,"-3,760,883,069","6,824,475,632","10,585,358,700",0,2025-05-26 00:09:03


In [17]:
''' Prepaid Revenue '''

v_prepaid_metric_list = ['B1R000100', 'TB1R000100', 'DB1R000100']
# v_tm_key_mth = 202501
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nPrepaid Revenue...\n')
print(f'   -> v_prepaid_metric_list: {v_prepaid_metric_list}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B1R000100_day_df = chk_src_df.copy()
B1R000100_day_df = B1R000100_day_df.loc[B1R000100_day_df['METRIC_CD'].isin(v_prepaid_metric_list)]
# B1R000100_day_df = B1R000100_day_df.loc[B1R000100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# B1R000100_day_df = B1R000100_day_df.loc[B1R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]

B1R000100_day_df = B1R000100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
B1R000100_day_df = B1R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

B1R000100_day_df['Diff (C - P)'] = B1R000100_day_df['C'] - B1R000100_day_df['P']
B1R000100_day_df = B1R000100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

mod_col_list = B1R000100_day_df.iloc[:, 4:8].columns.tolist()
for col in mod_col_list:
    B1R000100_day_df[col] = B1R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
B1R000100_day_df


Prepaid Revenue...

   -> v_prepaid_metric_list: ['B1R000100', 'TB1R000100', 'DB1R000100']


,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,Diff (C - P),C,P,G,PPN_TM
0,202501,Prepaid,B1R000100,Prepaid Revenue,"4,182,519","4,284,739,583","4,280,557,064","4,271,236,462",2025-05-26 03:06:46
1,202501,Prepaid,DB1R000100,Prepaid Revenue : DTAC,"-213,638","1,839,813,343","1,840,026,982","1,840,026,982",2025-05-26 03:06:46
2,202501,Prepaid,TB1R000100,Prepaid Revenue : TMH,"4,396,158","2,444,926,240","2,440,530,082","2,431,209,481",2025-05-26 03:06:46
3,202502,Prepaid,B1R000100,Prepaid Revenue,"5,697,778","3,865,905,070","3,860,207,291","3,818,091,848",2025-05-26 03:06:46
4,202502,Prepaid,DB1R000100,Prepaid Revenue : DTAC,"-172,089","1,648,646,062","1,648,818,151","1,648,818,151",2025-05-26 03:06:46
5,202502,Prepaid,TB1R000100,Prepaid Revenue : TMH,"5,869,867","2,217,259,007","2,211,389,140","2,169,273,696",2025-05-26 03:06:46
6,202503,Prepaid,B1R000100,Prepaid Revenue,"7,125,231","4,241,723,553","4,234,598,322","4,173,154,794",2025-05-26 03:06:46
7,202503,Prepaid,DB1R000100,Prepaid Revenue : DTAC,"-81,479","1,775,352,571","1,775,434,050","1,775,434,050",2025-05-26 03:06:46
8,202503,Prepaid,TB1R000100,Prepaid Revenue : TMH,"7,206,710","2,466,370,982","2,459,164,272","2,397,720,744",2025-05-26 03:06:46
9,202504,Prepaid,B1R000100,Prepaid Revenue,"6,750,686","4,079,845,439","4,073,094,754","4,062,564,046",2025-05-26 03:06:46


In [12]:
# ''' Postpaid Revenue '''

# v_postpaid_metric_list = ['B2R000100', 'TB2R000100', 'DB2R000100']
# # v_tm_key_mth = 202501
# # v_tm_key_day = int(str(curr_mth)+str('01'))
# # v_tm_key_day = 20250401
# print(f'\nPostpaid Revenue...\n')
# print(f'   -> v_postpaid_metric_list: {v_postpaid_metric_list}')
# # print(f'   -> v_tm_key_day: {v_tm_key_day}')

# B2R000100_day_df = chk_src_df.copy()
# B2R000100_day_df = B2R000100_day_df.loc[B2R000100_day_df['METRIC_CD'].isin(v_postpaid_metric_list)]
# # B2R000100_day_df = B2R000100_day_df.loc[B2R000100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# # B2R000100_day_df = B2R000100_day_df.loc[B2R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]

# B2R000100_day_df = B2R000100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
# B2R000100_day_df = B2R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

# B2R000100_day_df['Diff (C - P)'] = B2R000100_day_df['C'] - B2R000100_day_df['P']
# B2R000100_day_df = B2R000100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

# mod_col_list = B2R000100_day_df.iloc[:, 4:8].columns.tolist()
# for col in mod_col_list:
#     B2R000100_day_df[col] = B2R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
# B2R000100_day_df

In [14]:
# ''' Postpaid Revenue B2C '''

# v_postpaid_b2c_metric_list = ['B2R010100', 'TB2R010100', 'DB2R010100']
# # v_tm_key_mth = 202501
# # v_tm_key_day = int(str(curr_mth)+str('01'))
# # v_tm_key_day = 20250401
# print(f'\nPostpaid Revenue B2C...\n')
# print(f'   -> v_postpaid_b2c_metric_list: {v_postpaid_b2c_metric_list}')
# # print(f'   -> v_tm_key_day: {v_tm_key_day}')

# B2R010100_day_df = chk_src_df.copy()
# B2R010100_day_df = B2R010100_day_df.loc[B2R010100_day_df['METRIC_CD'].isin(v_postpaid_b2c_metric_list)]
# # B2R010100_day_df = B2R010100_day_df.loc[B2R010100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# # B2R010100_day_df = B2R010100_day_df.loc[B2R010100_day_df['TM_KEY_DAY']>=v_tm_key_day]

# B2R010100_day_df = B2R010100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
# B2R010100_day_df = B2R010100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

# B2R010100_day_df['Diff (C - P)'] = B2R010100_day_df['C'] - B2R010100_day_df['P']
# B2R010100_day_df = B2R010100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

# mod_col_list = B2R010100_day_df.iloc[:, 4:8].columns.tolist()
# for col in mod_col_list:
#     B2R010100_day_df[col] = B2R010100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
# B2R010100_day_df

In [18]:
''' TOL Revenue '''

v_tol_metric_list = ['TB3R000100']
# v_tm_key_mth = 202501
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTOL Revenue...\n')
print(f'   -> v_tol_metric_list: {v_tol_metric_list}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3R000100_day_df = chk_src_df.copy()
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['METRIC_CD'].isin(v_tol_metric_list)]
# TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]

TB3R000100_day_df = TB3R000100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
TB3R000100_day_df = TB3R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

TB3R000100_day_df['Diff (C - P)'] = TB3R000100_day_df['C'] - TB3R000100_day_df['P']
TB3R000100_day_df = TB3R000100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

mod_col_list = TB3R000100_day_df.iloc[:, 4:8].columns.tolist()
for col in mod_col_list:
    TB3R000100_day_df[col] = TB3R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3R000100_day_df


TOL Revenue...

   -> v_tol_metric_list: ['TB3R000100']


,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,Diff (C - P),C,P,G,PPN_TM
0,202501,TOL,TB3R000100,TOL Revenue,"4,545,013","1,597,056,733","1,592,511,720","1,592,511,720",2025-05-26 03:06:46
1,202502,TOL,TB3R000100,TOL Revenue,"4,254,776","1,609,673,918","1,605,419,142","1,605,419,142",2025-05-26 03:06:46
2,202503,TOL,TB3R000100,TOL Revenue,"4,185,979","1,616,442,724","1,612,256,745","1,612,256,745",2025-05-26 03:06:46
3,202504,TOL,TB3R000100,TOL Revenue,"4,079,823","1,616,821,911","1,612,742,089","1,612,742,089",2025-05-26 03:06:46
4,202505,TOL,TB3R000100,TOL Revenue,"3,377,722","1,272,938,901","1,269,561,179","1,269,561,179",2025-05-26 03:06:46


In [19]:
''' TVS Revenue '''

v_tvs_metric_list = ['TB4R000100']
# v_tm_key_mth = 202501
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTVS Revenue...\n')
print(f'   -> v_tvs_metric_list: {v_tvs_metric_list}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4R000100_day_df = chk_src_df.copy()
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['METRIC_CD'].isin(v_tvs_metric_list)]
# TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]

TB4R000100_day_df = TB4R000100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
TB4R000100_day_df = TB4R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

TB4R000100_day_df['Diff (C - P)'] = TB4R000100_day_df['C'] - TB4R000100_day_df['P']
TB4R000100_day_df = TB4R000100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

mod_col_list = TB4R000100_day_df.iloc[:, 4:8].columns.tolist()
for col in mod_col_list:
    TB4R000100_day_df[col] = TB4R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4R000100_day_df


TVS Revenue...

   -> v_tvs_metric_list: ['TB4R000100']


,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,Diff (C - P),C,P,G,PPN_TM
0,202501,TVS,TB4R000100,TVS Revenue,"124,433,738","417,316,338","292,882,600","292,882,600",2025-05-26 03:06:46
1,202502,TVS,TB4R000100,TVS Revenue,"14,211,771","307,227,824","293,016,052","293,016,052",2025-05-26 03:06:46
2,202503,TVS,TB4R000100,TVS Revenue,"-82,621,243","214,617,965","297,239,208","297,239,208",2025-05-26 03:06:46
3,202504,TVS,TB4R000100,TVS Revenue,"13,369,899","317,174,500","303,804,601","303,804,601",2025-05-26 03:06:46
4,202505,TVS,TB4R000100,TVS Revenue,"9,459,982","166,085,416","156,625,434","156,625,434",2025-05-26 03:06:46


## * SubBase & Reported SubBase

In [2]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000702' --Prepaid SubBase
        , 'DB1S000702' --Prepaid SubBase : DTAC
        , 'TB1S000702' --Prepaid SubBase : TMH
        , 'B2S010602' --Postpaid SubBase B2C
        , 'DB2S010602' --Postpaid SubBase B2C : DTAC
        , 'TB2S010602' --Postpaid SubBase B2C : TMH
        , 'TB3S020604' --FTTx SubBase
        , 'B1S000700' --Prepaid Reported SubBase
        , 'DB1S000700' --Prepaid Reported SubBase : DTAC
        , 'TB1S000700' --Prepaid Reported SubBase : TMH
        , 'B2S010600' --Postpaid Reported SubBase B2C
        , 'DB2S010600' --Postpaid Reported SubBase B2C : DTAC
        , 'TB2S010600' --Postpaid Reported SubBase B2C : TMH
        , 'TB3S000600' --FTTx Reported SubBase
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531, 20250630, 20250731, 20250831, 20250930, 20251031, 20251130, 20251231)
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-30, 10:43:39
   -> v_start_date: 20250101


In [3]:
''' Automate Month Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

#### Sample Data

In [4]:
sample_mth_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202501].copy()

sample_mth_df['METRIC_TMP'] = sample_mth_df['METRIC_NAME'].replace(r' : DTAC| : TRUE| : TMH', '', regex=True)
sample_mth_df['ALL'] = np.where(sample_mth_df['COMP_CD']=='ALL', sample_mth_df['P'], 0)
sample_mth_df['TRUE'] = np.where(sample_mth_df['COMP_CD']=='TRUE', sample_mth_df['P'], 0)
sample_mth_df['DTAC'] = np.where(sample_mth_df['COMP_CD']=='DTAC', sample_mth_df['P'], 0)

sample_mth_df = sample_mth_df.groupby(['TM_KEY_MTH', 'METRIC_TMP']).agg({'ACTUAL_AS_OF':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum', 'PPN_TM':'max'})
sample_mth_df = sample_mth_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_TMP']).reset_index()

mod_col_list = sample_mth_df.iloc[:, 3:6].columns.tolist()
for col in mod_col_list:
    sample_mth_df[col] = sample_mth_df[col].apply(lambda x: format(x, ',.0f'))

sample_mth_df

,TM_KEY_MTH,METRIC_TMP,ACTUAL_AS_OF,ALL,TRUE,DTAC,PPN_TM
0,202501,FTTx Reported SubBase,20250131,0,"3,163,192",0,2025-05-30 01:36:20
1,202501,FTTx SubBase,20250131,0,"3,280,710",0,2025-05-30 01:36:20
2,202501,Postpaid Reported SubBase B2C,20250131,"14,069,220","9,035,281","5,033,939",2025-05-30 01:36:20
3,202501,Postpaid SubBase B2C,20250131,"14,274,718","9,113,093","5,161,625",2025-05-30 01:36:20
4,202501,Prepaid Reported SubBase,20250131,0,"21,379,455",0,2025-05-30 01:36:20
5,202501,Prepaid SubBase,20250131,0,"24,942,269",0,2025-05-30 01:36:20


In [5]:
sample_day_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==20250131].copy()

mod_col_list = sample_day_df.iloc[:, 8:12].columns.tolist()
for col in mod_col_list:
    sample_day_df[col] = sample_day_df[col].apply(lambda x: format(x, ',.0f'))

sample_day_df = sample_day_df.sort_values(by=['TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_NAME']).reset_index(drop=True)
sample_day_df

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,ACTUAL_AS_OF,C,P,G,H,HH,PPN_TM
0,202501,20250131,Postpaid,ALL,B2S010600,Postpaid Reported SubBase B2C,20250131,None,"14,069,220","14,063,387","13,889,946","14,063,360",2025-05-30 01:36:20
1,202501,20250131,Postpaid,DTAC,DB2S010600,Postpaid Reported SubBase B2C : DTAC,20250131,None,"5,033,939","5,028,342","4,854,928","5,028,342",2025-05-30 01:36:20
2,202501,20250131,Postpaid,TRUE,TB2S010600,Postpaid Reported SubBase B2C : TMH,20250131,None,"9,035,281","9,035,045","9,035,018","9,035,018",2025-05-30 01:36:20
3,202501,20250131,Postpaid,ALL,B2S010602,Postpaid SubBase B2C,20250131,None,"14,274,718","9,112,857","9,112,830","9,112,830",2025-05-30 01:36:20
4,202501,20250131,Postpaid,DTAC,DB2S010602,Postpaid SubBase B2C : DTAC,20250131,None,"5,161,625",nan,nan,nan,2025-05-29 23:54:18
5,202501,20250131,Postpaid,TRUE,TB2S010602,Postpaid SubBase B2C : TMH,20250131,None,"9,113,093","9,112,857","9,112,830","9,112,830",2025-05-30 01:36:20
6,202501,20250131,Prepaid,TRUE,TB1S000700,Prepaid Reported SubBase : TMH,20250131,None,"21,379,455","21,262,196","21,262,196","21,262,196",2025-05-30 01:36:20
7,202501,20250131,Prepaid,TRUE,TB1S000702,Prepaid SubBase : TMH,20250131,None,"24,942,269","24,769,264","24,267,797","24,769,149",2025-05-30 01:36:20
8,202501,20250131,TOL,TRUE,TB3S000600,FTTx Reported SubBase,20250131,None,"3,163,192","3,163,109","3,163,109","3,163,109",2025-05-30 01:36:20
9,202501,20250131,TOL,TRUE,TB3S020604,FTTx SubBase,20250131,None,"3,280,710","3,280,625","3,280,625","3,280,625",2025-05-30 01:36:20


#### Product Summary

In [6]:
''' Prepaid SubBase '''

v_prepaid_subbase_metric_list = ['B1S000702', 'DB1S000702', 'TB1S000702']
# v_tm_key_mth = 202501
print(f'\nPrepaid SubBase...\n')
print(f'   -> metric_list: {v_prepaid_subbase_metric_list}')

B1S000702_day_df = chk_src_df.loc[chk_src_df['METRIC_CD'].isin(v_prepaid_subbase_metric_list)].copy()
# B1S000702_day_df = B1S000702_day_df.loc[B1S000702_day_df['TM_KEY_MTH']>=v_tm_key_mth]

B1S000702_day_df = B1S000702_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
B1S000702_day_df['Diff (P - G)'] = B1S000702_day_df['P'] - B1S000702_day_df['G']
B1S000702_day_df = B1S000702_day_df.sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD']).reset_index()

mod_col_list = B1S000702_day_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    B1S000702_day_df[col] = B1S000702_day_df[col].apply(lambda x: format(x, ',.0f'))
    
B1S000702_day_df


Prepaid SubBase...

   -> metric_list: ['B1S000702', 'DB1S000702', 'TB1S000702']


,TM_KEY_MTH,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,Diff (P - G)
0,202501,Prepaid,TRUE,TB1S000702,Prepaid SubBase : TMH,2025-05-30 01:36:20,0,"24,942,269","24,769,264","24,267,797","24,769,149","173,005"
1,202502,Prepaid,TRUE,TB1S000702,Prepaid SubBase : TMH,2025-05-30 01:36:20,0,"24,921,942","24,760,083","24,758,973","24,758,973","161,859"
2,202503,Prepaid,TRUE,TB1S000702,Prepaid SubBase : TMH,2025-05-30 01:36:20,0,"24,578,672","24,437,486","24,436,544","24,436,544","141,186"
3,202504,Prepaid,TRUE,TB1S000702,Prepaid SubBase : TMH,2025-05-30 01:36:20,0,"23,721,266","23,582,316","23,581,524","23,581,524","138,950"


In [7]:
''' Prepaid Reported SubBase '''

v_prepaid_reported_subbase_metric_list = ['B1S000700', 'DB1S000700', 'TB1S000700']
# v_tm_key_mth = 202501
print(f'\nPrepaid Reported SubBase...\n')
print(f'   -> metric_list: {v_prepaid_reported_subbase_metric_list}')

B1S000700_day_df = chk_src_df.loc[chk_src_df['METRIC_CD'].isin(v_prepaid_reported_subbase_metric_list)].copy()
# B1S000700_day_df = B1S000700_day_df.loc[B1S000700_day_df['TM_KEY_MTH']>=v_tm_key_mth]

B1S000700_day_df = B1S000700_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
B1S000700_day_df['Diff (P - G)'] = B1S000700_day_df['P'] - B1S000700_day_df['G']
B1S000700_day_df = B1S000700_day_df.sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD']).reset_index()

mod_col_list = B1S000700_day_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    B1S000700_day_df[col] = B1S000700_day_df[col].apply(lambda x: format(x, ',.0f'))
    
B1S000700_day_df


Prepaid Reported SubBase...

   -> metric_list: ['B1S000700', 'DB1S000700', 'TB1S000700']


,TM_KEY_MTH,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,Diff (P - G)
0,202501,Prepaid,TRUE,TB1S000700,Prepaid Reported SubBase : TMH,2025-05-30 01:36:20,0,"21,379,455","21,262,196","21,262,196","21,262,196","117,259"
1,202502,Prepaid,TRUE,TB1S000700,Prepaid Reported SubBase : TMH,2025-05-30 01:36:20,0,"21,170,310","21,078,525","21,078,525","21,078,525","91,785"
2,202503,Prepaid,TRUE,TB1S000700,Prepaid Reported SubBase : TMH,2025-05-30 01:36:20,0,"20,647,169","20,559,792","20,559,792","20,559,792","87,377"
3,202504,Prepaid,TRUE,TB1S000700,Prepaid Reported SubBase : TMH,2025-05-30 01:36:20,0,"20,304,003","20,209,194","20,209,194","20,209,194","94,809"


In [8]:
''' Postpaid SubBase B2C '''

v_postpaid_subbase_b2c_metric_list = ['B2S010602', 'DB2S010602', 'TB2S010602']
# v_tm_key_mth = 202501
print(f'\nPostpaid SubBase B2C...\n')
print(f'   -> metric_list: {v_postpaid_subbase_b2c_metric_list}')

B2S010602_day_df = chk_src_df.loc[chk_src_df['METRIC_CD'].isin(v_postpaid_subbase_b2c_metric_list)].copy()
# B2S010602_day_df = B2S010602_day_df.loc[B2S010602_day_df['TM_KEY_MTH']>=v_tm_key_mth]

B2S010602_day_df = B2S010602_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
B2S010602_day_df['Diff (P - G)'] = B2S010602_day_df['P'] - B2S010602_day_df['G']
B2S010602_day_df = B2S010602_day_df.sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD']).reset_index()

mod_col_list = B2S010602_day_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    B2S010602_day_df[col] = B2S010602_day_df[col].apply(lambda x: format(x, ',.0f'))
    
B2S010602_day_df


Postpaid SubBase B2C...

   -> metric_list: ['B2S010602', 'DB2S010602', 'TB2S010602']


,TM_KEY_MTH,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,Diff (P - G)
0,202501,Postpaid,ALL,B2S010602,Postpaid SubBase B2C,2025-05-30 01:36:20,0,"14,274,718","9,112,857","9,112,830","9,112,830","5,161,861"
1,202501,Postpaid,DTAC,DB2S010602,Postpaid SubBase B2C : DTAC,2025-05-29 23:54:18,0,"5,161,625",0,0,0,"5,161,625"
2,202501,Postpaid,TRUE,TB2S010602,Postpaid SubBase B2C : TMH,2025-05-30 01:36:20,0,"9,113,093","9,112,857","9,112,830","9,112,830",236
3,202502,Postpaid,ALL,B2S010602,Postpaid SubBase B2C,2025-05-30 01:36:20,0,"14,273,218","9,134,299","9,134,283","9,134,283","5,138,919"
4,202502,Postpaid,DTAC,DB2S010602,Postpaid SubBase B2C : DTAC,2025-05-29 23:54:18,0,"5,138,667",0,0,0,"5,138,667"
5,202502,Postpaid,TRUE,TB2S010602,Postpaid SubBase B2C : TMH,2025-05-30 01:36:20,0,"9,134,551","9,134,299","9,134,283","9,134,283",252
6,202503,Postpaid,ALL,B2S010602,Postpaid SubBase B2C,2025-05-30 01:36:20,0,"14,288,650","9,170,353","9,170,341",0,"5,118,297"
7,202503,Postpaid,DTAC,DB2S010602,Postpaid SubBase B2C : DTAC,2025-05-29 23:54:18,0,"5,118,045",0,0,0,"5,118,045"
8,202503,Postpaid,TRUE,TB2S010602,Postpaid SubBase B2C : TMH,2025-05-30 01:36:20,0,"9,170,605","9,170,353","9,170,341",0,252
9,202504,Postpaid,ALL,B2S010602,Postpaid SubBase B2C,2025-05-30 01:36:20,0,"14,301,830","9,194,222","9,194,211",0,"5,107,608"


In [9]:
''' Postpaid Reported SubBase B2C '''

v_postpaid_reported_subbase_b2c_metric_list = ['B2S010600', 'DB2S010600', 'TB2S010600']
# v_tm_key_mth = 202501
print(f'\nPostpaid Reported SubBase B2C...\n')
print(f'   -> metric_list: {v_postpaid_reported_subbase_b2c_metric_list}')

B2S010600_day_df = chk_src_df.loc[chk_src_df['METRIC_CD'].isin(v_postpaid_reported_subbase_b2c_metric_list)].copy()
# B2S010600_day_df = B2S010600_day_df.loc[B2S010600_day_df['TM_KEY_MTH']>=v_tm_key_mth]

B2S010600_day_df = B2S010600_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
B2S010600_day_df['Diff (P - G)'] = B2S010600_day_df['P'] - B2S010600_day_df['G']
B2S010600_day_df = B2S010600_day_df.sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD']).reset_index()

mod_col_list = B2S010600_day_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    B2S010600_day_df[col] = B2S010600_day_df[col].apply(lambda x: format(x, ',.0f'))
    
B2S010600_day_df


Postpaid Reported SubBase B2C...

   -> metric_list: ['B2S010600', 'DB2S010600', 'TB2S010600']


,TM_KEY_MTH,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,Diff (P - G)
0,202501,Postpaid,ALL,B2S010600,Postpaid Reported SubBase B2C,2025-05-30 01:36:20,0,"14,069,220","14,063,387","13,889,946","14,063,360","5,833"
1,202501,Postpaid,DTAC,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-05-30 01:36:20,0,"5,033,939","5,028,342","4,854,928","5,028,342","5,597"
2,202501,Postpaid,TRUE,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-05-30 01:36:20,0,"9,035,281","9,035,045","9,035,018","9,035,018",236
3,202502,Postpaid,ALL,B2S010600,Postpaid Reported SubBase B2C,2025-05-30 01:36:20,0,"14,068,955","14,063,110","14,063,094","14,063,094","5,845"
4,202502,Postpaid,DTAC,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-05-30 01:36:20,0,"5,013,920","5,008,326","5,008,326","5,008,326","5,594"
5,202502,Postpaid,TRUE,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-05-30 01:36:20,0,"9,055,035","9,054,784","9,054,768","9,054,768",251
6,202503,Postpaid,ALL,B2S010600,Postpaid Reported SubBase B2C,2025-05-30 01:36:20,0,"14,168,597","14,162,752","14,162,740","4,992,399","5,845"
7,202503,Postpaid,DTAC,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-05-30 01:36:20,0,"4,997,992","4,992,399","4,992,399","4,992,399","5,593"
8,202503,Postpaid,TRUE,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-05-30 01:36:20,0,"9,170,605","9,170,353","9,170,341",0,252
9,202504,Postpaid,ALL,B2S010600,Postpaid Reported SubBase B2C,2025-05-30 01:36:20,0,"14,182,881","14,177,004","14,176,993","4,982,782","5,877"


In [10]:
''' FTTx SubBase '''

v_fttx_subbase_metric_list = ['TB3S020604']
# v_tm_key_mth = 202501
print(f'\nFTTx SubBase...\n')
print(f'   -> metric_list: {v_fttx_subbase_metric_list}')

TB3S020604_day_df = chk_src_df.loc[chk_src_df['METRIC_CD'].isin(v_fttx_subbase_metric_list)].copy()
# TB3S020604_day_df = TB3S020604_day_df.loc[TB3S020604_day_df['TM_KEY_MTH']>=v_tm_key_mth]

TB3S020604_day_df = TB3S020604_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
TB3S020604_day_df['Diff (P - G)'] = TB3S020604_day_df['P'] - TB3S020604_day_df['G']
TB3S020604_day_df = TB3S020604_day_df.sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD']).reset_index()

mod_col_list = TB3S020604_day_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    TB3S020604_day_df[col] = TB3S020604_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3S020604_day_df


FTTx SubBase...

   -> metric_list: ['TB3S020604']


,TM_KEY_MTH,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,Diff (P - G)
0,202501,TOL,TRUE,TB3S020604,FTTx SubBase,2025-05-30 01:36:20,0,"3,280,710","3,280,625","3,280,625","3,280,625",85
1,202502,TOL,TRUE,TB3S020604,FTTx SubBase,2025-05-30 01:36:20,0,"3,290,380","3,290,298","3,290,298","3,290,298",82
2,202503,TOL,TRUE,TB3S020604,FTTx SubBase,2025-05-30 01:36:20,0,"3,305,177","3,305,095","3,305,095","3,305,095",82
3,202504,TOL,TRUE,TB3S020604,FTTx SubBase,2025-05-30 01:36:20,0,"3,315,573","3,315,491","3,315,491","3,315,491",82


In [11]:
''' FTTx Reported SubBase '''

v_fttx_reported_subbase_metric_list = ['TB3S000600']
# v_tm_key_mth = 202501
print(f'\nFTTx Reported SubBase...\n')
print(f'   -> metric_list: {v_fttx_reported_subbase_metric_list}')

TB3S000600_day_df = chk_src_df.loc[chk_src_df['METRIC_CD'].isin(v_fttx_reported_subbase_metric_list)].copy()
# TB3S000600_day_df = TB3S000600_day_df.loc[TB3S000600_day_df['TM_KEY_MTH']>=v_tm_key_mth]

TB3S000600_day_df = TB3S000600_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
TB3S000600_day_df['Diff (P - G)'] = TB3S000600_day_df['P'] - TB3S000600_day_df['G']
TB3S000600_day_df = TB3S000600_day_df.sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD']).reset_index()

mod_col_list = TB3S000600_day_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    TB3S000600_day_df[col] = TB3S000600_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3S000600_day_df


FTTx Reported SubBase...

   -> metric_list: ['TB3S000600']


,TM_KEY_MTH,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,Diff (P - G)
0,202501,TOL,TRUE,TB3S000600,FTTx Reported SubBase,2025-05-30 01:36:20,0,"3,163,192","3,163,109","3,163,109","3,163,109",83
1,202502,TOL,TRUE,TB3S000600,FTTx Reported SubBase,2025-05-30 01:36:20,0,"3,172,770","3,172,690","3,172,690","3,172,690",80
2,202503,TOL,TRUE,TB3S000600,FTTx Reported SubBase,2025-05-30 01:36:20,0,"3,187,717","3,187,637","3,187,637","3,187,637",80
3,202504,TOL,TRUE,TB3S000600,FTTx Reported SubBase,2025-05-30 01:36:20,0,"3,191,503","3,191,424","3,191,424","3,191,424",79
